# TODO
1. Import newspapers
2.
3. Translate to english
4. Find out which type of regression to use
5. Reduce text complexity processes
6. Perform topic modeling
7. Perform similarity comparison of topic models
8. Add similarity to df and perform regression

# Topic Modeling of Solar Renewable Energy Projects

# 1) Data

## 1a) Add DENA info to df

### Importing DENA files to a df

In [ ]:
import os # reads directory of stored DENA pdfs
from PyPDF2 import PdfReader # reads pdfs
import re # regex is used for finding date published of DENA report
import pandas as pd

directory  = 'DENA'
df = pd.DataFrame(columns = ["name", "year", "doc", "BETD", "DENA", "news", "txt_sim_news", "txt_sim_fce", "txt_sim_proj"])

for filename in os.listdir(directory): # for loop for importing all DENA files to a dataframe
    f = os.path.join(directory,filename)
    if os.path.isfile(f):
        reader = PdfReader(f)
        n_pages = reader._get_num_pages()
        n_pages = n_pages-1
        n_text = []
        document = ''

        for i in range(n_pages):
            page = reader.pages[n_pages]
            text = page.extract_text()
            document = document + text

        reg_date = re.findall('2017|2018|2019|2020|2021|2022', document)

        year = ''
        if reg_date:
            year  = reg_date[0]
        if not reg_date:
            reg_date = re.findall('2018|2019|2020|2021|2022', filename)
            if reg_date:
                year = reg_date[0]
            if not reg_date:
                year = 'na'
        filename = filename.rstrip('.pdf')
       # insert to df
        df.loc[-1] = [filename, year ,document, 0, 1, 0, 'na', 'na', 'na']
        df.index = df.index + 1
        df = df.sort_index()

# print(df['doc'].iloc[2])

In [20]:
from pdfminer.high_level import extract_text

directory = 'DENA'

for filename in os.listdir(directory):
    file = os.path.join(directory,filename)
    if os.path.isfile(file):
        document = extract_text(file)
        filename = filename.rstrip('.pdf')
        reg_date = re.findall('2017|2018|2019|2020|2021|2022', document)

        year = ''
        if reg_date:
            year  = reg_date[0]
        # insert to df
        df.loc[-1] = [filename, year ,document, 0, 1, 0, 'na', 'na', 'na']
        df.index = df.index + 1
        df = df.sort_index()

print(df)

                                                  name  year  \
0                           9173_RES-Projekt_Kolumbien         
1    Report_Status_and_Perspectives_of_Renewable_En...  2017   
2                               9168_RES-Projekt_Ghana  2020   
3    dena-Umfrage_Kuenstliche_Intelligenz_in_der_En...  2021   
4                              dena-Factsheet_RES_Togo  2019   
..                                                 ...   ...   
859            dena-ANALYSE_Marktmonitoring_Bioenergie    na   
860                                RES-Projekt_Ecuador  2019   
861  Abschlussbericht_dena-Leitstudie_Aufbruch_Klim...    na   
862           POSITIONSPAPIER_EEG-Novelle_Layout_final    na   
863  dena-Abschlussbericht_Wirksamer_Klimaschutz_du...    na   

                                                   doc  BETD  DENA  news  \
0    RES-PROJEkT kOLuMBIEn\ndena-Renewable-Energy-S...     0     1     0   
1    Report \nStatus and Perspectives of  \nRenewab...     0     1     0   
2  

### Cleaning DENA files

In [15]:
# filling in DENA information with additional text files
#todo: drop all columns that are not within date range after fixing files
# df.drop(axis=0, index=0, inplace=True)
# df['year'].iloc[112] = 2019
# df['doc'].iloc[112] = ''
# print(df['doc'].iloc[111])

In [16]:
# TEAM: make this cell to loop through df and manually add year at index, n=65

## 1b) Importing BETD files

In [17]:
directories = ['BETD18', 'BETD19', 'BETD21', 'BETD22']
year = 2018
for directory in directories:
    if year == 2020:
        year = year + 1
    for filename in os.listdir(directory):
        file = os.path.join(directory,filename)
        if os.path.isfile(file):
            with open(file) as f:
                document = f.readlines()
            filename = filename.rstrip('.pdf')

            # insert to df
            df.loc[-1] = [filename, year ,document, 1, 0, 0, 'na', 'na', 'na']
            df.index = df.index + 1
            df = df.sort_index()
    year = year + 1

## 1c) Importing newspapers

In [18]:
from pdfminer.high_level import extract_text

directory = 'newspapers'

for filename in os.listdir(directory):
    file = os.path.join(directory,filename)
    if os.path.isfile(file):
        document = extract_text(file)
        filename = filename.rstrip('.pdf')
        reg_date = re.findall('2017|2018|2019|2020|2021|2022', document)

        year = ''
        if reg_date:
            year  = reg_date[0]
        # insert to df
        df.loc[-1] = [filename, year ,document, 0, 0, 1, 'na', 'na', 'na']
        df.index = df.index + 1
        df = df.sort_index()

print(df)

                                                  name  year  \
0                        Solarbranche_in_Erklärungsnot  2021   
1                      Endspiel_für_die_Klimakanzlerin  2019   
2                                   Blick_in_die_Sonne  2022   
3    Die_allermeisten_Deutschen_haben_es_warm_genug...  2018   
4                               Sonne_mach_mich_reich_  2018   
..                                                 ...   ...   
746            dena-ANALYSE_Marktmonitoring_Bioenergie    na   
747                                RES-Projekt_Ecuador  2019   
748  Abschlussbericht_dena-Leitstudie_Aufbruch_Klim...    na   
749           POSITIONSPAPIER_EEG-Novelle_Layout_final    na   
750  dena-Abschlussbericht_Wirksamer_Klimaschutz_du...    na   

                                                   doc  BETD  DENA  news  \
0    Solarbranche in Erklärungsnot\n\nHandelsblatt ...     1     0     0   
1    Endspiel für die Klimakanzlerin\n\nHandelsblat...     1     0     0   
2  

## 1d) Detect language in df cell and translate inplace

In [19]:
# DeepL api integration (must have account for api to run cell)
# print(df['doc'].iloc[111])